In [2]:
test_ct_only= '/home/nada.saadi/CTPET/hecktor2022_cropped/test_ct_only_mda.json'
test_pet_only= '/home/nada.saadi/CTPET/hecktor2022_cropped/test_pet_only_mda.json'
test_CTPT= '/home/nada.saadi/CTPET/hecktor2022_cropped/test_CTPT_only_mda.json'

In [3]:
Encoder_frozen_PT_weights='/home/nada.saadi/CTPET/hecktor2022_cropped/module2-with-pet-skipconnection-firsthalf/m2-withpet-intheskipconnection-firsthalf.pth'
Encoder_decoder_frozen_PT_weights= '/home/nada.saadi/CTPET/hecktor2022_cropped/module2-with-pet-skipconnection-firsthalf-decoderfrozen/m2-withpet-intheskipconnection-firsthalf-decoderfrozen.pth'

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from collections.abc import Sequence

from unetr import CustomedUNETR

import json
from tqdm.autonotebook import tqdm
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 

print(torch.cuda.device_count()) 
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

1


/tmp/ipykernel_1215655/2558345750.py:9: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
from monai.transforms import EnsureTyped
from monai.transforms import Compose, LoadImaged, ScaleIntensityRanged, ConcatItemsd
from monai.metrics import DiceMetric
from monai.losses import DiceCELoss, DiceFocalLoss, FocalLoss
from monai.inferers import sliding_window_inference


from monai.data import (
    Dataset,
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)


from monai.transforms import (
    AsDiscrete,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    MapTransform,
    ScaleIntensityd,
    #AddChanneld,
    SpatialPadd,
    CenterSpatialCropd,
    EnsureChannelFirstd,
    ConcatItemsd,
    AdjustContrastd, 
    Rand3DElasticd,
    HistogramNormalized,
    NormalizeIntensityd,
    Invertd,
    SaveImage,

)

In [6]:
data_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped'
json_dir=test_ct_only

In [28]:
model = CustomedUNETR(
    in_channels=1,
    out_channels=3,
    img_size=(96, 96, 96),
    feature_size=48,
    hidden_size=768,
    num_heads=12,
    mlp_dim=3072,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
    proj_type="conv",
    use_ct_encoder=True,  # Disable CT encoder
    use_pet_encoder=False,  # Disable PET encoder for CT-only inference
    use_lora=False,  # Disable LoRA
    
    
).to(device)

/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: unetr CustomedUNETR.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.
  warn_deprecated(argname, msg, warning_category)


Nada's version of UNETR
(96, 96, 96)
(16, 16, 16)
(6, 6, 6)
768
zaz w dakchi ya s lbnat


In [7]:
model

CustomedUNETR(
  (vit): ViT(
    (patch_embedding): PatchEmbeddingBlock(
      (patch_embeddings): Conv3d(1, 768, kernel_size=(16, 16, 16), stride=(16, 16, 16))
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (blocks): ModuleList(
      (0-11): 12 x TransformerBlock(
        (mlp): MLPBlock(
          (linear1): Linear(in_features=768, out_features=3072, bias=True)
          (linear2): Linear(in_features=3072, out_features=768, bias=True)
          (fn): GELU(approximate='none')
          (drop1): Dropout(p=0.0, inplace=False)
          (drop2): Dropout(p=0.0, inplace=False)
        )
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): SABlock(
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_q): Linear(in_features=768, out_features=768, bias=False)
          (proj_k): Linear(in_features=768, out_features=768, bias=False)
          (proj_v): Linear(in_features=768, out_features=768, bias=False)
      

In [8]:
model.state_dict().keys()

odict_keys(['vit.patch_embedding.position_embeddings', 'vit.patch_embedding.patch_embeddings.weight', 'vit.patch_embedding.patch_embeddings.bias', 'vit.blocks.0.mlp.linear1.weight', 'vit.blocks.0.mlp.linear1.bias', 'vit.blocks.0.mlp.linear2.weight', 'vit.blocks.0.mlp.linear2.bias', 'vit.blocks.0.norm1.weight', 'vit.blocks.0.norm1.bias', 'vit.blocks.0.attn.out_proj.weight', 'vit.blocks.0.attn.out_proj.bias', 'vit.blocks.0.attn.proj_q.weight', 'vit.blocks.0.attn.proj_k.weight', 'vit.blocks.0.attn.proj_v.weight', 'vit.blocks.0.norm2.weight', 'vit.blocks.0.norm2.bias', 'vit.blocks.1.mlp.linear1.weight', 'vit.blocks.1.mlp.linear1.bias', 'vit.blocks.1.mlp.linear2.weight', 'vit.blocks.1.mlp.linear2.bias', 'vit.blocks.1.norm1.weight', 'vit.blocks.1.norm1.bias', 'vit.blocks.1.attn.out_proj.weight', 'vit.blocks.1.attn.out_proj.bias', 'vit.blocks.1.attn.proj_q.weight', 'vit.blocks.1.attn.proj_k.weight', 'vit.blocks.1.attn.proj_v.weight', 'vit.blocks.1.norm2.weight', 'vit.blocks.1.norm2.bias', 'vi

In [6]:
class ClipCT(MapTransform):
    """
    Convert labels to multi channels based on hecktor classes:
    label 1 is the tumor
    label 2 is the lymph node

    """

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            if key == "ct":
                d[key] = torch.clip(d[key], min=-200, max=200)
            # elif key == "pt":
            #     d[key] = torch.clip(d[key], d[key].min(), 5)
        return d

class MulPTFM(MapTransform):
    """
    Mult PT and FM 

    """

    def __call__(self, data):
        d = dict(data)

        fm = d["ct"] > 0
        d["pt"] = d["pt"] * fm
        return d

class SelectClass(MapTransform):
    """
    Select the class for which you want to fine tune the model 

    """
    # def __init__(self, keys, cls=1):
    #     super(self).__init__(keys)
    #     self.cls = cls

    def __call__(self, data):
        d = dict(data)
        d["seg"][d["seg"] == 1] = 0
        # d["seg"][d["seg"] == 2] = 1
        
        return d

In [7]:
def datafold_read(datalist, basedir, fold=0):
    with open(datalist) as f:
        json_data = json.load(f)

    val = []
    for d in json_data:
        if "fold" in d and d["fold"] == fold and "key" not in d:
            for k in d:
                if isinstance(d[k], list):
                    d[k] = [os.path.join(basedir, iv) for iv in d[k]]
                elif isinstance(d[k], str):
                    d[k] = os.path.join(basedir, d[k]) if len(d[k]) > 0 else d[k]
            val.append(d)

    return val


In [10]:
import json

# Load the file
with open('/home/nada.saadi/CTPET/hecktor2022_cropped/test_pet_only_mda.json', 'r') as f:
    data = json.load(f)

# Print the first dictionary
print(data[0])


{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-005', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-005/MDA-005_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-005/MDA-005_gt.nii.gz', 'fold': 0}


In [11]:
modified_valid_data = datafold_read(datalist=json_dir, basedir=data_dir, fold=0)
len(modified_valid_data)

44

In [13]:
# val_transforms = Compose(
#     [
#         LoadImaged(keys=[ "pt", "seg"], ensure_channel_first = True),
#         SpatialPadd(keys=[ "pt", "seg"], spatial_size=(200, 200, 310), method='end'),
#         Orientationd(keys=[ "pt", "seg"], axcodes="PLS"),
#         NormalizeIntensityd(keys=["pt"]),
#         ClipCT(keys=["pt"]),
#         ScaleIntensityd(keys=["pt"], minv=0, maxv=1),
#         #MulPTFM(keys=["ct","pt"]),
#         ConcatItemsd(keys=["pt"], name="pt"),
#     ]
# )
# def create_dataloader(data, transforms, batch_size=2, shuffle=True):
#     # Create CacheDataset with the reformatted data
#     dataset = Dataset(data=data, transform=transforms)

#     # Create DataLoader
#     return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=8)
# val_loader = create_dataloader(modified_valid_data, val_transforms, shuffle=False)

In [9]:
val_transforms = Compose(
    [
        LoadImaged(keys=["ct",  "seg"], ensure_channel_first = True),
        SpatialPadd(keys=["ct",  "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=["ct",  "seg"], axcodes="PLS"),
        #NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["ct"]),
        ScaleIntensityd(keys=["ct",], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        ConcatItemsd(keys=[ "ct"], name="ct"),
    ]
)
def create_dataloader(data, transforms, batch_size=2, shuffle=True):
    # Create CacheDataset with the reformatted data
    dataset = Dataset(data=data, transform=transforms)

    # Create DataLoader
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=8)
val_loader = create_dataloader(ct_data, val_transforms, shuffle=False)

In [10]:
torch.backends.cudnn.benchmark = True
def poly_lr(epoch, max_epochs, initial_lr, exponent=0.9):
    return initial_lr * (1 - epoch / max_epochs)**exponent

loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [16]:
ckpt= torch.load(Encoder_frozen_PT_weights)
ckpt.keys()

odict_keys(['vit.lora_vit.patch_embedding.position_embeddings', 'vit.lora_vit.patch_embedding.patch_embeddings.weight', 'vit.lora_vit.patch_embedding.patch_embeddings.bias', 'vit.lora_vit.blocks.0.mlp.linear1.weight', 'vit.lora_vit.blocks.0.mlp.linear1.bias', 'vit.lora_vit.blocks.0.mlp.linear2.weight', 'vit.lora_vit.blocks.0.mlp.linear2.bias', 'vit.lora_vit.blocks.0.norm1.weight', 'vit.lora_vit.blocks.0.norm1.bias', 'vit.lora_vit.blocks.0.attn.out_proj.weight', 'vit.lora_vit.blocks.0.attn.out_proj.bias', 'vit.lora_vit.blocks.0.attn.proj_q.w.weight', 'vit.lora_vit.blocks.0.attn.proj_q.w_a.weight', 'vit.lora_vit.blocks.0.attn.proj_q.w_b.weight', 'vit.lora_vit.blocks.0.attn.proj_k.weight', 'vit.lora_vit.blocks.0.attn.proj_v.w.weight', 'vit.lora_vit.blocks.0.attn.proj_v.w_a.weight', 'vit.lora_vit.blocks.0.attn.proj_v.w_b.weight', 'vit.lora_vit.blocks.0.norm2.weight', 'vit.lora_vit.blocks.0.norm2.bias', 'vit.lora_vit.blocks.1.mlp.linear1.weight', 'vit.lora_vit.blocks.1.mlp.linear1.bias', 'v

In [17]:
def load_weights(ckpt):
    new_ckpt={}
    for key in ckpt.keys():
        if "lora" in key:
            new_key = key.replace("lora_vit.","")
            new_ckpt[new_key] = ckpt[key]
        else:
            new_ckpt[key] = ckpt[key]
            
    return new_ckpt

In [18]:
ckpt= torch.load(Encoder_frozen_PT_weights)
# new_ckpt = load_weights(ckpt)
model.load_state_dict(ckpt)

RuntimeError: Error(s) in loading state_dict for CustomedUNETR:
	Missing key(s) in state_dict: "vit.patch_embedding.position_embeddings", "vit.patch_embedding.patch_embeddings.weight", "vit.patch_embedding.patch_embeddings.bias", "vit.blocks.0.mlp.linear1.weight", "vit.blocks.0.mlp.linear1.bias", "vit.blocks.0.mlp.linear2.weight", "vit.blocks.0.mlp.linear2.bias", "vit.blocks.0.norm1.weight", "vit.blocks.0.norm1.bias", "vit.blocks.0.attn.out_proj.weight", "vit.blocks.0.attn.out_proj.bias", "vit.blocks.0.attn.proj_q.weight", "vit.blocks.0.attn.proj_k.weight", "vit.blocks.0.attn.proj_v.weight", "vit.blocks.0.norm2.weight", "vit.blocks.0.norm2.bias", "vit.blocks.1.mlp.linear1.weight", "vit.blocks.1.mlp.linear1.bias", "vit.blocks.1.mlp.linear2.weight", "vit.blocks.1.mlp.linear2.bias", "vit.blocks.1.norm1.weight", "vit.blocks.1.norm1.bias", "vit.blocks.1.attn.out_proj.weight", "vit.blocks.1.attn.out_proj.bias", "vit.blocks.1.attn.proj_q.weight", "vit.blocks.1.attn.proj_k.weight", "vit.blocks

In [ ]:
Encoder_frozen_PT_weights = '/home/nada.saadi/CTPET/hecktor2022_cropped/module2-with-pet-skipconnection-firsthalf-decoderfrozen/m2-withpet-intheskipconnection-firsthalf-decoderfrozen.pth'
msg = model.load_state_dict(torch.load(Encoder_frozen_PT_weights), strict=False)
print(msg)

<All keys matched successfully>


In [ ]:
#def sliding_window_inference_fix(inputs, roi_size, sw_batch_size, model):#
    # Calculate the padding required to make the input size a multiple of the roi_size
    padding = [0, 0, 0]
    for i in range(3):
        if inputs.shape[i + 2] % roi_size[i] != 0:
            padding[i] = roi_size[i] - (inputs.shape[i + 2] % roi_size[i])

    # Pad the inputs if necessary
    if sum(padding) > 0:
        pad_fn = nn.ConstantPad3d(padding, 0)
        inputs = pad_fn(inputs)

# Perform sliding window inference
    outputs = []
    for i in range(0, inputs.shape[2] - roi_size[0] + 1, roi_size[0]):
        for j in range(0, inputs.shape[3] - roi_size[1] + 1, roi_size[1]):
            for k in range(0, inputs.shape[4] - roi_size[2] + 1, roi_size[2]):
                input_patch = inputs[:, :, i:i+roi_size[0], j:j+roi_size[1], k:k+roi_size[2]]
                output_patch = model(input_patch)
                outputs.append(output_patch)

    # Remove padding from the outputs
    if sum(padding) > 0:
        outputs = outputs[:-(padding[0] * padding[1] * padding[2])]

    return torch.cat(outputs, dim=0)

IndentationError: unexpected indent (1799742331.py, line 3)

In [30]:
from monai.transforms import apply_transform
from monai.inferers import sliding_window_inference
from monai.transforms import AsDiscrete
from monai.data import DataLoader, Dataset, decollate_batch
from tqdm import tqdm
import json
import torch

# Load the pre-trained weights
# Encoder_frozen_PT_weights ='/home/nada.saadi/CTPET/hecktor2022_cropped/module2-cttokens/m2-cttokens.pth'
# #Encoder_frozen_PT_weights = '/home/nada.saadi/CTPET/hecktor2022_cropped/module2-with-pet-skipconnection-firsthalf/m2-withpet-intheskipconnection-firsthalf.pth'
# #Encoder_frozen_PT_weights = '/home/nada.saadi/CTPET/hecktor2022_cropped/module2-with-pet-skipconnection-firsthalf-decoderfrozen/m2-withpet-intheskipconnection-firsthalf-decoderfrozen.pth'
# msg = model.load_state_dict(torch.load(Encoder_frozen_PT_weights), strict=False)
# print(msg)

# Load the pre-trained weights
# Encoder_frozen_PT_weights = '/home/nada.saadi/CTPET/hecktor2022_cropped/Experiments with Lora/MDA-CTPT-LORA-ENCODERFROZEN/mda-ctpt-loraencoderfrozen.pth'

# # Load the pre-trained weights only for the matching layers

# Load the pre-trained weights only for the matching layers
# state_dict = torch.load(Encoder_frozen_PT_weights)
# model_dict = model.state_dict()
# matched_state_dict = {k: v for k, v in state_dict.items() if k in model_dict}

# # Fix the size mismatch for encoder1.layer.conv1.conv.weight and encoder1.layer.conv3.conv.bias
# matched_state_dict['encoder1.layer.conv1.conv.weight'] = matched_state_dict['encoder1.layer.conv1.conv.weight'][:, :1, :, :, :]
# matched_state_dict['encoder1.layer.conv3.conv.weight'] = matched_state_dict['encoder1.layer.conv3.conv.weight'][:, :1, :, :, :]  

# model_dict.update(matched_state_dict)
# model.load_state_dict(model_dict)

# Assume `test_ct_only` is your test CT only dataset loaded from a JSON or other sources
# For demonstration, let's say `test_ct_only` is available as a list of dictionaries
# Each dictionary contains paths to the 'ct', 'pt', and 'seg' for each test sample

# Assume `val_transforms` is already defined and includes all necessary preprocessing steps

# test_loader = DataLoader(
#     dataset=Dataset(data=modified_valid_data, transform=val_transforms),
#     batch_size=1,  # Batch size for testing can be 1 since no backpropagation is required
#     shuffle=False,
#     num_workers=4
# )
import io
from monai.transforms import AsDiscrete
from monai.data import DataLoader, Dataset, decollate_batch
ct_data='/home/nada.saadi/CTPET/hecktor2022_cropped/test_ct_only_mda.json'
with open(ct_data, 'r') as f:
    test_data = json.load(f)

test_loader = DataLoader(
    dataset=Dataset(data=test_data, transform=val_transforms),
    batch_size=1,  # Batch size for testing can be 1 since no backpropagation is required
    shuffle=False,
    num_workers=4
)
loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

post_label = AsDiscrete(to_onehot=3)
post_pred = AsDiscrete(argmax=True, to_onehot=3)

# Dice metric for evaluation
dice_metric_fn = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
dice_metric_batch_fn = DiceMetric(include_background=False, reduction="mean_batch")

ctweights= '/home/nada.saadi/CTPET/hecktor2022_cropped/module2-with-pet-skipconnection-firsthalf-decoderfrozen/m2-withpet-intheskipconnection-firsthalf-decoderfrozen.pth'
model.load_state_dict(torch.load(ctweights), strict=False)

def testing():
    model.eval()
    dice_metric_fn.reset()
    dice_metric_batch_fn.reset()
    with torch.no_grad():
        for batch_data in tqdm(test_loader):
            test_inputs, test_labels = batch_data["ct"].cuda(), batch_data["seg"].cuda()
            
            test_outputs = sliding_window_inference(test_inputs, (96, 96, 96), 4, model)
            
            # Convert outputs and labels to one-hot format for DiceMetric calculation
            test_outputs = [AsDiscrete(argmax=True, to_onehot=3)(i) for i in decollate_batch(test_outputs)]
            test_labels = [AsDiscrete(to_onehot=3)(i) for i in decollate_batch(test_labels)]

            dice_metric_fn(y_pred=test_outputs, y=test_labels)
            dice_metric_batch_fn(y_pred=test_outputs, y=test_labels)

    mean_dice_test = dice_metric_fn.aggregate().item()
    metric_batch_test = dice_metric_batch_fn.aggregate()
    metric_tumor = metric_batch_test[0].item()
    metric_lymph = metric_batch_test[1].item()

    print(f"Testing - Avg Dice: {mean_dice_test:.4f}, Tumor Dice: {metric_tumor:.4f}, Lymph Dice: {metric_lymph:.4f}")
    return mean_dice_test, metric_tumor, metric_lymph
    # Save the weights
save_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/testing_ct_saved_weights.pth'
torch.save(model.state_dict(), save_path)
print(f"Model weights saved at: {save_path}")


testing()


Model weights saved at: /home/nada.saadi/CTPET/hecktor2022_cropped/testing_ct_saved_weights.pth


  0%|          | 0/44 [00:02<?, ?it/s]


RuntimeError: shape '[4, 6, 6, 6, 768]' is invalid for input of size 331776

In [ ]:
# import torch
# from monai.inferers import sliding_window_inference
# from monai.transforms import AsDiscrete
# from monai.metrics import DiceMetric
# from monai.data import DataLoader, Dataset, decollate_batch
# from tqdm import tqdm
# import json


# # Load the pre-trained weights
# pretrained_weights_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/module2-with-pet-skipconnection-firsthalf/m2-withpet-intheskipconnection-firsthalf.pth'
# model.load_state_dict(torch.load(pretrained_weights_path), strict=True)

# # Assume `valid_data` is your validation dataset loaded from a JSON or other sources
# # For demonstration, let's say `valid_data` is available as a list of dictionaries
# # Each dictionary contains paths to the 'ct', 'pt', and 'seg' for each validation sample

# # Assume `val_transforms` is already defined and includes all necessary preprocessing steps
# test_ct_only = '/home/nada.saadi/CTPET/hecktor2022_cropped/test_ct_only_mda.json'


# val_loader = DataLoader(
#     dataset=Dataset(data=test_ct_only, transform=val_transforms),
#     batch_size=1,  # Batch size for validation can be 1 since no backpropagation is required
#     shuffle=False,
#     num_workers=4
# )

# # Dice metric for evaluation
# dice_metric_fn = DiceMetric(include_background=False, reduction="mean")
# dice_metric_batch_fn = DiceMetric(include_background=False, reduction="mean_batch")

# def validation():
#     model.eval()
#     dice_metric_fn.reset()
#     dice_metric_batch_fn.reset()
#     with torch.no_grad():
#         for batch_data in tqdm(val_loader,fold=0):
#             val_inputs, val_labels = batch_data["ct"].cuda(), batch_data["seg"].cuda()
#             val_outputs = sliding_window_inference(val_inputs, (96, 96, 96), 4, model)
            
#             # Convert outputs and labels to one-hot format for DiceMetric calculation
#             val_outputs = [AsDiscrete(argmax=True, to_onehot=3)(i) for i in decollate_batch(val_outputs)]
#             val_labels = [AsDiscrete(to_onehot=3)(i) for i in decollate_batch(val_labels)]

#             dice_metric_fn(y_pred=val_outputs, y=val_labels)
#             dice_metric_batch_fn(y_pred=val_outputs, y=val_labels)

#     mean_dice_val = dice_metric_fn.aggregate().item()
#     metric_batch_val = dice_metric_batch_fn.aggregate()
#     metric_tumor = metric_batch_val[0].item()
#     metric_lymph = metric_batch_val[1].item()

#     print(f"Validation - Avg Dice: {mean_dice_val:.4f}, Tumor Dice: {metric_tumor:.4f}, Lymph Dice: {metric_lymph:.4f}")

# # Run validation
# validation()


In [ ]:
def check_data_format(data):
    if not isinstance(data, list):
        print("Data is not a list.")
        return False

    for i, item in enumerate(data):
        if not isinstance(item, dict):
            print(f"Item at index {i} is not a dictionary.")
            return False
        if len(item) != 2:
            print(f"Dictionary at index {i} does not have exactly two key-value pairs.")
            return False

    print("Data format is correct.")
    return True

# Check the format of valid_data
check_data_format(valid_data)

NameError: name 'valid_data' is not defined

In [ ]:
def create_modified_data(data):
    modified_data = []
    for item in data:
        modified_item = {key: item[key] for key in ['ct', 'seg']}
        modified_data.append(modified_item)
    return modified_data

# Create a modified version of valid_data
modified_valid_data = create_modified_data(valid_data)

# Check the format of modified_valid_data
check_data_format(modified_valid_data)

Data format is correct.


True

In [ ]:
from monai.transforms import apply_transform
test_sample = {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001/MDA-001_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001/MDA-001_gt.nii.gz', 'fold': 0}
transformed_sample = apply_transform(val_transforms, test_sample)
print(transformed_sample)


{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001', 'ct': metatensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.00

In [ ]:
import json

test_ct_only = '/home/nada.saadi/CTPET/hecktor2022_cropped/test_ct_only_mda.json'

with open(test_ct_only) as file:
    data = json.load(file)

first_5_elements = data[:5]
print(first_5_elements)


[{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001/MDA-001_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001/MDA-001_gt.nii.gz', 'fold': 0}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-007', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-007/MDA-007_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-007/MDA-007_gt.nii.gz', 'fold': 0}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-015', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-015/MDA-015_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-015/MDA-015_gt.nii.gz', 'fold': 0}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-018', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-018/MDA-018_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-018/MDA-018_gt.nii.gz', 'fold': 0}, {'id': 

In [ ]:
from monai.data import Dataset
from tqdm import tqdm

# Assuming test_ct_only is a list of dictionaries as shown in your JSON snippet
dataset = Dataset(data=test_ct_only, transform=val_transforms)

# Manually iterate over the dataset and apply transforms
for i in tqdm(range(len(dataset))):
    sample = dataset[i]  # This should not raise an error if everything is correct
    print(sample.keys())  # Should print dict keys including 'ct' and 'seg'


  0%|          | 0/64 [00:00<?, ?it/s]

RuntimeError: applying transform <monai.transforms.compose.Compose object at 0x7fbf900077f0>

In [ ]:
sample = dataset[0]
print(sample)


RuntimeError: applying transform <monai.transforms.compose.Compose object at 0x7fbf900077f0>

In [ ]:
simple_transforms = Compose([LoadImaged(keys=["ct", "seg"], ensure_channel_first=True)])
simple_dataset = Dataset(data=test_ct_only, transform=simple_transforms)
sample = simple_dataset[0]
print(sample)


RuntimeError: applying transform <monai.transforms.compose.Compose object at 0x7fc0e961c070>